<h1>Práctica 1 PySpark</h1>  

<h5>Autor: Guillermo García López<h5>

# Ejercicios básicos

<h3>1. Calcular la media y la desviación típica de un fichero con alturas (alturas_v0.csv). Cada fila tiene
una altura (en cm). Sin embargo, algunas filas tienen errores y pone -100. Asimismo, en algunas
filas las alturas están en metros en vez de en centímetros. Antes de calcular la media y la
desviación, hay que filtrar los datos erróneos y corregir los datos en otras unidades.</h3>

<h6>Resolución:</h6>

Creamos Spark Context:

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

Se importa el fichero "alturas_v0.csv":

In [2]:
alturasv0 = sc.textFile("alturas_v0.csv")

Se transforma el fichero, casteando cada dato a tipo float, y se filtra el fichero: sólo se seleccionarán 
elementos != -100.0:

In [3]:
_alturasv00 = alturasv0.map(lambda el: float(el))                                 \
                   .filter(lambda el: el != -100.0)

Se crea un RDD con los datos que vienen correctos (en cm):

In [4]:
_alturasv01 = _alturasv00.filter(lambda el: el >= 100)

Se crea otro RDD con los datos que vienen en metros (< 100), se transforma a cm:

In [5]:
_alturasv02 = _alturasv00.filter(lambda el: el < 100)           \
                     .map(lambda el: el*100)

Se unen ambos RDDs:

In [6]:
_alturasv0 = _alturasv01.union(_alturasv02)

Se calcula la media y desviación estándar de los datos:

In [10]:
media = _alturasv0.reduce(lambda el1, el2: el1 + el2) / _alturasv0.count()
desvest = _alturasv0.reduce(lambda el1, el2: el1 + el2) / _alturasv0.count()
print("Media de alturas: {0:5.4f}".format(media))
print("Desviación de alturas: {0:5.4f}".format(desvest))

Media de alturas: 168.9383
Desviación de alturas: 168.9383


<hr>

<h3>2. Calcular la media y la desviación típica de un fichero con alturas (alturas.csv) por género. Cada
    fila tiene género y altura (en cm). Se presentan los mismos errores que en el caso precedente.
    Transformaciones y acciones sugeridas: textFile, map, reduce, float(str) (Convierte una cadena a
    float), filter, count, split, etc.</h3>

<h6>Resolución:</h6>

Se importa el fichero "alturas.csv":

In [14]:
alturas = sc.textFile("alturas.csv")

Se transforma el fichero para crear una lista de tuplas ('género', altura),y se filtra el fichero: sólo se 
seleccionarán elementos != -100.0:

In [15]:
_alturas1 = alturas.map(lambda el: el.split(','))  \
                   .map(lambda el: (el[0], float(el[1]) ))  \
                   .filter(lambda el: el[1] != -100.0)

Se crea un RDD con los datos que vienen correctos (en cm):

In [16]:
_alturas2 = _alturas1.filter(lambda el: el[1] >= 100.0)

Se crea otro RDD con los datos que vienen en metros (< 100), se transforma a cm:

In [17]:
_alturas3 = _alturas1.filter(lambda el: el[1] < 100.0)           \
                     .map(lambda el: (el[0], el[1]*100))

Se unen ambos RDDs:

In [18]:
_alturas = _alturas1.union(_alturas2)

A partir de este RDD se crea uno para hombres y otro para mujeres, seleccionando únicamente los valores numéricos:

In [19]:
_alturasH = _alturas.filter(lambda el: el[0] == 'H')\
                    .map(lambda el: el[1])
_alturasM = _alturas.filter(lambda el: el[0] != 'H')\
                    .map(lambda el: el[1])

Y se calculan media y desviación para cada RDD:

In [20]:
sumaH  =  _alturasH.reduce(lambda el1, el2: el1+el2)
countH = _alturasH.count()
meanH  = sumaH / countH
auxH = _alturasH.map(lambda el: (el-meanH)**2)\
                .reduce(lambda el1, el2: el1 + el2)
stdH = ((1/countH) * auxH)**(1/2)
print("Media de Hombres: {0:5.4f} | Desviación de Hombres: {1:5.4f}".format(meanH, stdH))

Media de Hombres: 169.5991 | Desviación de Hombres: 26.4927


In [21]:
sumaM =  _alturasM.reduce(lambda el1, el2: el1+el2)
countM = _alturasM.count()
meanM  = sumaM / countM
auxM = _alturasM.map(lambda el: (el-meanM)**2)\
                .reduce(lambda el1, el2: el1 + el2)
stdM = ((1/countM) * auxM)**(1/2)
print("Media de Mujeres: {0:5.4f} | Desviación de Mujeres: {1:5.4f}".format(meanM, stdM))

Media de Mujeres: 159.8910 | Desviación de Mujeres: 26.5151
